<a href="https://colab.research.google.com/github/skyprince999/100-Days-Of-ML/blob/master/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters()) # learnable parameters of the model
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0807,  0.1883,  0.0030,  0.1156,  0.0911,  0.1073,  0.0934,  0.0993,
         -0.0723,  0.0545]], grad_fn=<AddmmBackward>)


In [0]:
net.zero_grad() # zero-fy all the gradients

out.backward(torch.randn(1, 10)) # populate all bckward prop parameters with random weights

torch.nn only supports mini-batches. The entire torch.nn package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, nn.Conv2d will take in a 4D Tensor of nSamples x nChannels x Height x Width.

If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.

In [7]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0120, grad_fn=<MseLossBackward>)


In [8]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [9]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0015, -0.0038,  0.0121,  0.0010,  0.0097,  0.0069])


In [0]:
learning_rate = 0.01 # Updating the weights using SGD - Stochastic Gradient descent 
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [12]:
for f in net.parameters():
  print(f)

Parameter containing:
tensor([[[[ 0.0098,  0.2013, -0.2911],
          [-0.0832, -0.0513, -0.1137],
          [-0.2934, -0.0344,  0.2736]]],


        [[[ 0.1649, -0.0928,  0.3074],
          [-0.2847, -0.0518,  0.3307],
          [-0.1339, -0.0758,  0.1598]]],


        [[[-0.0085,  0.0567, -0.0637],
          [-0.2711,  0.2255,  0.2951],
          [-0.2680, -0.2923, -0.1485]]],


        [[[ 0.3205,  0.3329, -0.3125],
          [-0.2410, -0.0199,  0.0375],
          [ 0.2822, -0.2193, -0.1531]]],


        [[[-0.2384, -0.0442, -0.0415],
          [-0.2063,  0.0297, -0.1050],
          [-0.2053, -0.0027, -0.0978]]],


        [[[ 0.0547, -0.0086, -0.1587],
          [-0.0122, -0.0067,  0.2708],
          [ 0.1789, -0.1853, -0.2696]]]], requires_grad=True)
Parameter containing:
tensor([-0.2036, -0.3057, -0.0980, -0.1694,  0.1465, -0.1779],
       requires_grad=True)
Parameter containing:
tensor([[[[-8.7797e-02,  2.8548e-02, -6.1985e-03],
          [ 1.0192e-01,  1.0771e-01,  1.7038e-02

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [0]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update